# UK Traffic Accidents ML Model Interpretability

<h1>Tabla de Contenidos<span class="tocSkip"></span></h1>
<br>
<div class="toc">
   <ul class="toc-item">
      <li>
         <span><a href="#Problem-Definition" data-toc-modified-id="Problem-Definition-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Problem Overview</a></span>
       </li>
       <li>
         <span><a href="#Dataset-Analysis" data-toc-modified-id="Dataset-Analysis-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Dataset</a></span>
         <ul class="toc-item">
            <li><span><a href="#Importing-Python-libraries" data-toc-modified-id="Importing-Python-libraries-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Importing Python libraries</a></span></li>
            <li><span><a href="#Importing-the-data" data-toc-modified-id="Importing-the-data-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Importing the data</a></span></li>
            <li>
               <span><a href="#Data-Description-and-Exploratory-Visualisations" data-toc-modified-id="Data-Description-and-Exploratory-Visualisations-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Dataset Analysis</a></span>
            </li>
             </li>
            <li>
<span><a href="#Building-Machine-Learning-Models" data-toc-modified-id="Building-Machine-Learning-Models-4"><span class="toc-item-num">3&nbsp;&nbsp;</span>Building Machine Learning Models</a></span>
            </li>
             <li>
                <span><a href="#Building-Machine-Learning-Models" data-toc-modified-id="Building-Machine-Learning-Models-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Interpreting the Machine Learning Models</a></span>
         <ul class="toc-item">
            <li><span><a href="#Baseline-Algorithms" data-toc-modified-id="Baseline-Algorithms-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Logistic Regression</a></span></li>
              <li>
               <span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Gradient Boosted Tree</a></span>
            </li>
             </ul>
             </li>
             </li>
      <li>
         <span><a href="#Concluding-Remarks" data-toc-modified-id="Concluding-Remarks-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Concluding Remarks</a></span>
       </li>
</div>

## 1. Problem Overview

Partimos de 3 .csv que guardan los datos de los accidentes, victimas y vehiculos de UK del año 2020. Fueron descargados desde la pagina oficial del gobierno de Gran Bretaña https://data.gov.uk/dataset/cb7ae6f0-4be6-4935-9277-47e5ce24a11f/road-safety-data. De esos 3 .csv iniciales, los unificamos en uno solo mediante el identificador del accidente y después (ya solo con un .csv) filtramos y obtenemos los datos que nos interesan para luego implementar los modelos. Una vez con los modelos implementados, realizamos la interpretación mediante las librerías correspondientes.

In [ ]:
from IPython import display
display.Image('data/Overview_practica3.png')

## 2. Dataset

En este caso de estudio, se ha obtenido un conjunto de datos de Road Safety Data de [Find Open Data](https://data.gov.uk/)  <br>

Como se indica en el [Sitio web del Dataset](https://data.gov.uk/dataset/cb7ae6f0-4be6-4935-9277-47e5ce24a11f/road-safety-data) 
<em>Estos archivos proporcionan datos detallados de seguridad vial sobre las circunstancias de los accidentes de tráfico con lesiones personales en Gran Bretaña desde 1979 (aunque nosotros usaremos únicamente los más actuales disponibles, los de 2020) los tipos de vehículos implicados y las víctimas resultantes. Las estadísticas se refieren únicamente a los accidentes con lesiones personales en la vía pública que se notifican a la policía, y que posteriormente se registran, utilizando el formulario de notificación de accidentes STATS19.<br>Todas las variables de datos están codificadas en lugar de contener cadenas textuales. Las tablas de búsqueda están disponibles en la sección "Recursos adicionales" hacia la parte inferior de la tabla.También incluye: Los resultados de los datos de las pruebas de alcoholemia de los dispositivos digitales de pruebas de alcoholemia introducidos recientemente, proporcionados por las autoridades policiales de Inglaterra y Gales Los resultados de los niveles de alcohol en sangre (miligramos / 100 mililitros de sangre) proporcionados por el cotejo de los datos de los médicos forenses (proporcionados por los médicos forenses de Inglaterra y Gales y por los Procuradores Fiscales de Escocia) con los datos de víctimas mortales de los datos policiales STATS19 de accidentes de tráfico en Gran Bretaña. Los casos en los que los niveles de alcohol en sangre de una víctima mortal son "desconocidos" son consecuencia de una coincidencia infructuosa entre los dos conjuntos de datos.</em>


### 2.1. Importing Python Libraries

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
%matplotlib inline
from IPython.display import display

from sklearn.model_selection import train_test_split
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve
from sklearn.metrics import auc, roc_auc_score, roc_curve, recall_score, log_loss
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, make_scorer
from sklearn.metrics import average_precision_score

from sklearn.linear_model import LogisticRegression

from sklearn.tree import DecisionTreeRegressor                                             
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text                
from sklearn.metrics import accuracy_score, classification_report , plot_confusion_matrix

from sklearn import svm
from mlxtend.plotting import plot_decision_regions

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler

from xgboost import XGBClassifier
from numpy import loadtxt
from xgboost import XGBClassifier
from xgboost import plot_tree
import matplotlib.pyplot as plt

import lime
import lime.lime_tabular

import eli5
from eli5.sklearn import PermutationImportance

from skater.core.explanations import Interpretation
from skater.model import InMemoryModel

### 2.2. Importing the data

Importamos el conjuntos de datos, contamos con 3 archivos CSV que previamente unificaremos en uno solo con las variables que nos interesen para proceder con el estudio.

In [ ]:
#Read csv file
df_vehicles = pd.read_csv('data/dft-road-casualty-statistics-vehicle-2020.csv')
df_accidents = pd.read_csv('data/dft-road-casualty-statistics-accident-2020.csv')
df_casualties = pd.read_csv('data/dft-road-casualty-statistics-casualty-2020.csv')
#Shape of CSV
print("El primer CSV contiene: {} (filas x columnas)".format(df_vehicles.shape))
print("El segundo CSV contiene: {} (filas x columnas)".format(df_accidents.shape))
print("El tercer CSV contiene: {} (filas x columnas)".format(df_casualties.shape))

Unificamos los 3 csv anteriores usando merge function con how='inner'. Dado el propio funcionamiento de la funcion lo deberemos hacer en dos partes (df_vehicles + df_accidents) + df_casualties

In [ ]:
#Unificamos df_accidents con df_vehicles
df_traffic_accidents_v1 = pd.merge(df_accidents, df_vehicles, how='inner')

#Unificamos df_traffic_accidents_v1 con df_casualties
df_traffic_accidents = pd.merge(df_traffic_accidents_v1, df_casualties, how='inner')
print("El archivo CSV final contiene: {} (filas x columnas)".format(df_traffic_accidents.shape))

In [ ]:
df_traffic_accidents.dropna(['date', 'time', 'accident_index', 'day_of_week', 'generic_make_model', 'car_passenger','did_police_officer_attend_scene_of_accident','sex_of_casualty','age_of_vehicle','age_of_driver','generic_make_model'] == -1)

In [ ]:
df_traffic_accidents = df_traffic_accidents.reindex(columns = ['accident_severity', 'number_of_vehicles', 'number_of_casualties', 'day_of_week', 'did_police_officer_attend_scene_of_accident', 'sex_of_driver', 'age_of_driver', 'age_of_vehicle', 'sex_of_casualty', 'age_of_casualty', 'car_passenger', 'casualty_severity'])

In [ ]:
df_traffic_accidents_lg = df_traffic_accidents.drop(['number_of_casualties','day_of_week', 'did_police_officer_attend_scene_of_accident',
                                                    'sex_of_driver','age_of_vehicle','age_of_driver', 'car_passenger'], axis=1)

### 2.3. Dataset Analysis

## 3. Building ML Models

In [ ]:
modelos = ["Logistic regression", "Decision Tree", "Gradient bossted tree", "Support Vector Machine"]
clasificadores = [LogisticRegression(solver = 'lbfgs', max_iter = 1000), 
                  DecisionTreeClassifier(criterion = 'entropy', max_depth = 3), svm.SVC(kernel='linear'), XGBClassifier()]
scores = []

## 4. Interpreting the Machine Learning Models

### 4.1. Logistic Regression

La regresion logistica es un modelo parametrico dado que ofrece un primer nivel de interpretacion a través de sus coeficientes. Primero usamos la libreria LIME para deducir que variables influyen más para clasificar la gravedad de la victima en las 3 opciones posibles (1, 2, 3). Veremos cuando influye más cada una y cuál es para la regresion logistica la más importante. 

In [ ]:
import lime
import lime.lime_tabular

Definimos las clases en las que la regresion logistica clasifica y predice para nuestra varible.

In [ ]:
classes = ['1','2','3']

Creamos el explainer, concretamente un Lime Tabular Explainer, donde le pasamos el X_train usado para entrenar el modelo (tanto los valores como las columnas) y luego las classes en las que clasifica. 

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(lr_X_train.values, feature_names=lr_X_train.columns, 
                                                   class_names=classes, kernel_width=5)

Recordamos como es el df usado, con el cual la regresion logistica realizara las predicciones:

In [ ]:
df_traffic_accidents_lg.head(1)

In [ ]:
lr_X_test

Para construir el modelo con LIME necesitamos una funcion de prediccion. El predict_proba nos dará la probabilidad predecida, despues lo convertimos a float y los asignamos a la funcion.

In [ ]:
predict_fn_nn= lambda x: lr.predict_proba(x).astype(float)

In [ ]:
predict_fn_nn

Para obtener el resultado le pasamos la instancia elegida y la funcion de prediccion hecha antes. Con el num_features le decimos que use 4 variables para realizar el analisis. Y despues mostramos los resultados. En este caso hacemos la prueba con la fila 62236 de nuestro conjunto de datos de test.

In [ ]:
lr_X_test.loc[5787]

In [ ]:
instancia_n = lr_X_test.loc[[5787]].values[0]
exp = explainer.explain_instance(instancia_n, predict_fn_nn, num_features=4)
exp.show_in_notebook(show_all=False)
exp.as_list()

Observamos que el resultado de la prediccion fue 2 (accidente severo) con 0.53 y que para el modelo el numero de vehiculos fue la variable mas importante para obtener la predicción, por delante de otras variables a priori vitales como la edad o el sexo.

Cuando realizamos el modelo de regresion logistica, pusimos varios casos hipotéticos y despues intentamos deducir que variables eran mas importantes, ahora podremos obtener la propia explicacion de las predicciones generadas y comparar los resultados. 

### Interpretabilidad Caso 1

En los dos primeros casos vamos a ver la importancia que tiene number_of_vehicles. Para accidentes  de 1 o 2 lo más probable es obtener un 2 como máximo mientras que para accidentes de 3 es poco habitual obtener valores como 1 o 2. En el primer caso, para una mujer de 65 con un accidente fuerte y 1 coche involucrado moriria en el momento mientras que para un hombre de 16 años en un accidente grave con 6 coches involucrados sería hospitalizado. 

In [ ]:
lr.predict(np.array([[1,1,2,65]]))[0]

In [ ]:
instancia_1 = np.array([[1,1,2,65]])[0]
exp = explainer.explain_instance(instancia_1, predict_fn_nn, num_features=4, top_labels=1)
exp.show_in_notebook(show_table = True, show_all=False)

In [ ]:
lr.predict(np.array([[1,6,2,65]]))[0]

In [ ]:
instancia_2 = np.array([[1,6,2,65]])[0]
exp = explainer.explain_instance(instancia_2, predict_fn_nn, num_features=4, top_labels=1)
exp.show_in_notebook(show_table = True, show_all=False)

#### Conclusion Caso 1

Vemos que para ambos casos el numero de vehiculos ocupa una posicion muy alta para definir la prediccion del modelo. El numero de vehiculos afecta notablemente en la severidad de la victima dado que como podemos ver, para un accidente con 1 vehiculo involucrado habría 0.77 posibilidades de morir mientras que con 6 coches habria 0.27 (Los dos casos para un accidente mortal, de una mujer de 65 años). En cambio, pensamos en un primer momento que la severidad del accidente era muy importante para clasificar dado que si un accidente fue muy grave no es muy habitual que la victima salga ilesa. No obstante, la variable accident_severity no resulta ser importante dado que tanto en este como en los siguientes ejemplos ocupa la ultima posicion. 

***Acertamos en:*** Relevancia de la variable number_of_vehicles. 

### Interpretabilidad Caso 2

En estos dos siguientes casos nos centraremos en las variables edad y sexo de la victima dado que van a resultar ser de vital importancia. En ambos casos pondremos la situación de un accidente "grave" con 3 coches involucrados y mientras que en uno se trata de un mujer de 65 y en otro de un hombre de 35. En el primer caso sería un accidente mortal mientras que el segundo sería hospitalizado. 

In [ ]:
lr.predict(np.array([[1,3,1,35]]))[0]

In [ ]:
instancia_3 = np.array([[1,3,1,35]])[0]
exp = explainer.explain_instance(instancia_3, predict_fn_nn, num_features=4, top_labels=1)
exp.show_in_notebook(show_table = True, show_all=False)

In [ ]:
lr.predict(np.array([[1,3,2,65]]))[0]

In [ ]:
instancia_4 = np.array([[1,3,2,65]])[0]
exp = explainer.explain_instance(instancia_4, predict_fn_nn, num_features=4, top_labels=1)
exp.show_in_notebook(show_table = True, show_all=False)

#### Conclusion Caso 2

Para este caso pensamos que la edad y el sexo eran importantes para la predicción, y así fue. No obstante, el numero de vehiculos sigue siendo más esencial que el sexo dado que es mucho más decisivo en el accidente. La edad de la victima se muestra como vital dado que afecta en la posibilidad de haber obtenido en los casos anteriores un accidente de grado 2 en vez de el mortal. Por ejemplo, con una edad de 35 la probabilidad de haber tenido un accidente menos severo es del 0.45 mientras que con casi el dosble de edad se reduce hasta el 0.42.

***Acertamos en:*** Relevancia de la variable age_of_casualty. 

### Interpretabilidad Caso 3

Y por ultimo veremos para unos datos de accidente identicos en cuanto a la victima como influye la severidad del accidente.

In [ ]:
lr.predict(np.array([[3,6,2,65]]))[0]

In [ ]:
instancia_5 = np.array([[3,6,2,65]])[0]
exp = explainer.explain_instance(instancia_5, predict_fn_nn, num_features=4, top_labels=1)
exp.show_in_notebook(show_table = True, show_all=False)

In [ ]:
lr.predict(np.array([[1,6,2,65]]))[0]

In [ ]:
instancia_5 = np.array([[1,6,2,65]])[0]
exp = explainer.explain_instance(instancia_5, predict_fn_nn, num_features=4, top_labels=1)
exp.show_in_notebook(show_table = True, show_all=False)

#### Conclusion Caso 3

Para ambos casos se trata de un accidente con 6 vehiculos involucrados, de una mujer con 65 años. La variable accident_severity sigue siendo poco importante para la predicción pero en el caso en el que el resto de variables hipoteticas sean iguales permite clasificar de diferente manera. En el caso de un accidente de grado 3, habría un 100% de resultar ileso mientras que en un accidente de 1, no habría ninguna. Estaría repartido con un 1/3 entre 1 y 2. 

***Acertamos en:*** Relevancia de la variable accident_severity pero unicamente en el caso en el que todas las variables sean iguales ella sera la definidora final de la prediccion. 

### 4.2 Gradient Boosted Tree

## 5. Concluding remarks